In [5]:
!make gen-server

# Compile protos
pip install grpcio-tools==1.51.1 mypy-protobuf==3.4.0 'types-protobuf>=3.20.4' --no-cache-dir
mkdir text_generation_server/pb || true
mkdir: cannot create directory ‘text_generation_server/pb’: File exists
python -m grpc_tools.protoc -I../proto --python_out=text_generation_server/pb \
	--grpc_python_out=text_generation_server/pb --mypy_out=text_generation_server/pb ../proto/generate.proto
/opt/conda/lib/python3.9/site-packages/grpc_tools/protoc.py:21: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  import pkg_resources
Writing mypy to generate_pb2.pyi
find text_generation_server/pb/ -type f -name "*.py" -print0 -exec sed -i -e 's/^\(import.*pb2\)/from . \1/g' {} \;
text_generation_server/pb/generate_pb2.py text_generation_server/pb/generate_pb2_grpc.py text_generation_server/pb/__init__.py touch text_generation_server/pb/__init__.py


In [6]:
%cd text_generation_server
%ls

/usr/src/server-dev/text_generation_server


__init__.py   cache.py  interceptor.py  pb/        tracing.py
__pycache__/  cli.py    models/         server.py  utils/


In [7]:
from text_generation_server.models.flash_llama import FlashLlama

model = FlashLlama(model_id="meta-llama/Llama-2-7b-hf")

You are using a model of type llama to instantiate a model of type . This is not supported for all configurations of models and can yield errors.


In [10]:
model.model.model

FlashLlamaModel(
  (embed_tokens): TensorParallelEmbedding()
  (layers): ModuleList(
    (0-31): 32 x FlashLlamaLayer(
      (self_attn): FlashLlamaAttention(
        (rotary_emb): PositionRotaryEmbedding()
        (query_key_value): TensorParallelColumnLinear(
          (linear): FastLinear()
        )
        (o_proj): TensorParallelRowLinear(
          (linear): FastLinear()
        )
      )
      (mlp): LlamaMLP(
        (act): SiLUActivation()
        (gate_up_proj): TensorParallelColumnLinear(
          (linear): FastLinear()
        )
        (down_proj): TensorParallelRowLinear(
          (linear): FastLinear()
        )
      )
      (input_layernorm): LlamaRMSNorm()
      (post_attention_layernorm): LlamaRMSNorm()
    )
  )
  (norm): LlamaRMSNorm()
)

In [11]:
from peft.mapping import (
    MODEL_TYPE_TO_PEFT_MODEL_MAPPING,
    PEFT_TYPE_TO_CONFIG_MAPPING,
)

PEFT_TYPE_TO_CONFIG_MAPPING["LORA"]

peft.tuners.lora.LoraConfig

In [13]:
from peft.peft_model import PeftModelForCausalLM
from peft.tuners.lora import LoraConfig

lora_id = "nealchandra/llama-2-7b-hf-lora-alpaca-json"

lora_config = LoraConfig.from_pretrained(lora_id)
lora_config

LoraConfig(peft_type='LORA', auto_mapping=None, base_model_name_or_path='/models/meta-llama--Llama-2-7b-hf', revision=None, task_type='CAUSAL_LM', inference_mode=True, r=16, target_modules=['q_proj', 'k_proj', 'v_proj', 'o_proj'], lora_alpha=16, lora_dropout=0.05, fan_in_fan_out=False, bias='none', modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None)

In [24]:
from text_generation_server.utils.layers import (
    TensorParallelColumnLinear,
    TensorParallelRowLinear
)

FUSED_MODULES = ["q_proj", "k_proj", "v_proj"]

def find_and_replace(model, lora_config):
    assert lora_config.bias == "none"
   
    target_modules_non_fused = [
        module for module in lora_config.target_modules
        if module not in FUSED_MODULES
    ]

    target_modules_fused = [
        module for module in lora_config.target_modules
        if module in FUSED_MODULES
    ]

class BLoraTensorParallelColumnLinear(TensorParallelColumnLinear):

class BLoraTensorParallelRowLinear:


def create_new_module(model, lora_config, target):
    bias = hasattr(target, "bias") and target.bias is not None
    kwargs = {
        "r": lora_config.r,
        "lora_alpha": lora_config.lora_alpha,
        "lora_dropout": lora_config.lora_dropout,
        "fan_in_fan_out": lora_config.fan_in_fan_out,
        "init_lora_weights": lora_config.init_lora_weights,
    }

    if isinstance(target, TensorParallelColumnLinear):
        pass
    elif isinstance(target, TensorParallelRowLinear):
        pass
    else:
        raise ValueError(
            f"Target module {target} is not supported. "
            f"Currently, only `TensorParallelColumnLinear` and `TensorParallelRowLinear` are supported."
        )
    


In [35]:
from peft.tuners.lora import LoraModel
from peft.utils.other import _get_submodules

key_list = [key for key, _ in model.model.named_modules()]

for key in key_list:
    if not LoraModel._check_target_module_exists(None, lora_config, key):
        pass
    else:
        parent, target, target_name = _get_submodules(model.model, key)
        print(parent)
        print(target)
        print(target_name)
        assert False

FlashLlamaAttention(
  (rotary_emb): PositionRotaryEmbedding()
  (query_key_value): TensorParallelColumnLinear(
    (linear): FastLinear()
  )
  (o_proj): TensorParallelRowLinear(
    (linear): FastLinear()
  )
)
TensorParallelRowLinear(
  (linear): FastLinear()
)
o_proj


AssertionError: 

In [22]:
lora_config.target_modules

['q_proj', 'k_proj', 'v_proj', 'o_proj']

In [ ]:
from text_generation_server.models import FlashLlama
from typing import Dict

class BLoraFlashLlama(FlashLlama):
    def __init__(
        model,
        config
    ):
        
        